In [19]:
import numpy as np
import cv2
import dlib
from sklearn.cluster import KMeans
import math
from math import degrees

In [20]:
imagepath = "sample.jpg"
# link = https://github.com/opencv/opencv/tree/master/data/haarcascades
face_cascade_path = "data/haarcascade_frontalface_default.xml"
#.dat file for detecting facial landmarks
#download file path = http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
predictor_path = "data/shape_predictor_68_face_landmarks.dat"

In [21]:
faceCascade = cv2.CascadeClassifier(face_cascade_path)
predictor = dlib.shape_predictor(predictor_path)
image = cv2.imread(imagepath)
image = cv2.resize(image, (500, 500))
original = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gauss = cv2.GaussianBlur(gray,(3,3), 0)

In [22]:
faces = faceCascade.detectMultiScale(
    gauss,
    scaleFactor=1.05,
    minNeighbors=5,
    minSize=(100,100),
    flags=cv2.CASCADE_SCALE_IMAGE
    )
print("found {0} faces!".format(len(faces)) )

found 1 faces!


In [23]:
for (x,y,w,h) in faces:
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
    detected_landmarks = predictor(image, dlib_rect).parts()
    landmarks = np.matrix([[p.x,p.y] for p in detected_landmarks])
results = original.copy()

In [24]:
for (x,y,w,h) in faces:
    cv2.rectangle(results, (x,y), (x+w,y+h), (0,255,0), 2)
    temp = original.copy()
    forehead = temp[y:y+int(0.25*h), x:x+w]
    rows,cols, bands = forehead.shape
    X = forehead.reshape(rows*cols,bands)
   
    kmeans = KMeans(n_clusters=2,init='k-means++',max_iter=300,n_init=10, random_state=0)
    y_kmeans = kmeans.fit_predict(X)
    for i in range(0,rows):
        for j in range(0,cols):
            if y_kmeans[i*cols+j]==True:
                forehead[i][j]=[255,255,255]
            if y_kmeans[i*cols+j]==False:
                forehead[i][j]=[0,0,0]
    
    forehead_mid = [int(cols/2), int(rows/2) ] 
    lef=0 
    pixel_value = forehead[forehead_mid[1],forehead_mid[0] ]
    for i in range(0,cols):
        if forehead[forehead_mid[1],forehead_mid[0]-i].all()!=pixel_value.all():
            lef=forehead_mid[0]-i
            break;
    left = [lef,forehead_mid[1]]
    rig=0
    for i in range(0,cols):
        if forehead[forehead_mid[1],forehead_mid[0]+i].all()!=pixel_value.all():
            rig = forehead_mid[0]+i
            break;
    right = [rig,forehead_mid[1]]

In [25]:
line1 = np.subtract(right+y,left+x)[0]
cv2.line(results, tuple(x+left), tuple(y+right), color=(0,255,0), thickness = 2)
cv2.putText(results,' Line 1',tuple(x+left),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
cv2.circle(results, tuple(x+left), 5, color=(255,0,0), thickness=-1)    
cv2.circle(results, tuple(y+right), 5, color=(255,0,0), thickness=-1)        

array([[[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       ...,

       [[206, 208, 208],
        [206, 208, 208],
        [206, 208, 208],
        ...,
        [192, 207, 223],
        [195, 211, 224],
        [197, 213, 226]],

       [[203, 205, 205],
        [204, 206, 206],
        [204, 206, 206],
        ...,
        [193, 208, 224],
        [197, 213, 226],
        [196, 212, 225]],

       [[201, 203, 203],
        [202, 204, 204],
        [202, 204, 204],
        ...,
        [196, 211, 227],
        [199, 215, 228],
        [198, 214, 227]]

In [26]:
linepointleft = (landmarks[1,0],landmarks[1,1])
linepointright = (landmarks[15,0],landmarks[15,1])
line2 = np.subtract(linepointright,linepointleft)[0]
cv2.line(results, linepointleft,linepointright,color=(0,255,0), thickness = 2)
cv2.putText(results,' Line 2',linepointleft,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
cv2.circle(results, linepointleft, 5, color=(255,0,0), thickness=-1)    
cv2.circle(results, linepointright, 5, color=(255,0,0), thickness=-1)

array([[[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       ...,

       [[206, 208, 208],
        [206, 208, 208],
        [206, 208, 208],
        ...,
        [192, 207, 223],
        [195, 211, 224],
        [197, 213, 226]],

       [[203, 205, 205],
        [204, 206, 206],
        [204, 206, 206],
        ...,
        [193, 208, 224],
        [197, 213, 226],
        [196, 212, 225]],

       [[201, 203, 203],
        [202, 204, 204],
        [202, 204, 204],
        ...,
        [196, 211, 227],
        [199, 215, 228],
        [198, 214, 227]]

In [27]:
linepointleft = (landmarks[3,0],landmarks[3,1])
linepointright = (landmarks[13,0],landmarks[13,1])
line3 = np.subtract(linepointright,linepointleft)[0]
cv2.line(results, linepointleft,linepointright,color=(0,255,0), thickness = 2)
cv2.putText(results,' Line 3',linepointleft,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
cv2.circle(results, linepointleft, 5, color=(255,0,0), thickness=-1)    
cv2.circle(results, linepointright, 5, color=(255,0,0), thickness=-1)

array([[[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       ...,

       [[206, 208, 208],
        [206, 208, 208],
        [206, 208, 208],
        ...,
        [192, 207, 223],
        [195, 211, 224],
        [197, 213, 226]],

       [[203, 205, 205],
        [204, 206, 206],
        [204, 206, 206],
        ...,
        [193, 208, 224],
        [197, 213, 226],
        [196, 212, 225]],

       [[201, 203, 203],
        [202, 204, 204],
        [202, 204, 204],
        ...,
        [196, 211, 227],
        [199, 215, 228],
        [198, 214, 227]]

In [28]:
linepointbottom = (landmarks[8,0],landmarks[8,1])
linepointtop = (landmarks[8,0],y)
line4 = np.subtract(linepointbottom,linepointtop)[1]
cv2.line(results,linepointtop,linepointbottom,color=(0,255,0), thickness = 2)
cv2.putText(results,' Line 4',linepointbottom,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
cv2.circle(results, linepointtop, 5, color=(255,0,0), thickness=-1)    
cv2.circle(results, linepointbottom, 5, color=(255,0,0), thickness=-1)

array([[[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       [[208, 210, 210],
        [208, 210, 210],
        [208, 210, 210],
        ...,
        [206, 208, 208],
        [205, 207, 207],
        [205, 207, 207]],

       ...,

       [[206, 208, 208],
        [206, 208, 208],
        [206, 208, 208],
        ...,
        [192, 207, 223],
        [195, 211, 224],
        [197, 213, 226]],

       [[203, 205, 205],
        [204, 206, 206],
        [204, 206, 206],
        ...,
        [193, 208, 224],
        [197, 213, 226],
        [196, 212, 225]],

       [[201, 203, 203],
        [202, 204, 204],
        [202, 204, 204],
        ...,
        [196, 211, 227],
        [199, 215, 228],
        [198, 214, 227]]

In [29]:
similarity = np.std([line1,line2,line3])
ovalsimilarity = np.std([line2,line4])

In [30]:
ax,ay = landmarks[3,0],landmarks[3,1]
bx,by = landmarks[4,0],landmarks[4,1]
cx,cy = landmarks[5,0],landmarks[5,1]
dx,dy = landmarks[6,0],landmarks[6,1]

In [31]:
alpha0 = math.atan2(cy-ay,cx-ax)
alpha1 = math.atan2(dy-by,dx-bx)
alpha = alpha1-alpha0
angle = abs(degrees(alpha))
angle = 180-angle

In [32]:
for i in range(1):
    if similarity<10:
        if angle<160:
            print('Squared shape. Jawlines are more angular')
            break
        else:
            print('Round shape. Jawlines are not that angular')
            break
    if line3>line1:
        if angle<160:
            print('Triangle shape.Forehead is more wider') 
            break
    if ovalsimilarity<10:
        print('Diamond shape. Line2 & line4 are similar and line2 is slightly larger')
        break
    if line4 > line2:
        if angle<160:
            print('Rectangular. Face length is largest and jawline are angular ')
            break
        else:
            print('Oblong. Face length is largest and jawlines are not angular')
            break


Oblong. Face length is largest and jawlines are not angular


In [33]:
output = np.concatenate((original,results), axis=1)
cv2.imwrite('output.jpg',output)

True